Red Neuronal Pruebas

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Cargar los datos
data = pd.read_csv("sample_data/diabetes.csv")

# Normalizar los datos
scaler = StandardScaler()
data[data.columns[:-1]] = scaler.fit_transform(data[data.columns[:-1]])

# Convertir a tensores de PyTorch
X = torch.tensor(data.drop('Outcome', axis=1).values, dtype=torch.float32)
y = torch.tensor(data['Outcome'].values, dtype=torch.float32).view(-1, 1)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir la arquitectura del modelo
class DiabetesNN(nn.Module):
    def __init__(self):
        super(DiabetesNN, self).__init__()
        self.fc1 = nn.Linear(8, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Instanciar el modelo
model = DiabetesNN()

# Definir la función de pérdida y el optimizador
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Tamaño de lote, número de épocas y tasa de aprendizaje
batch_size = 64
num_epochs = 10
learning_rate = 0.001

# Entrenamiento del modelo
train_losses = []
test_losses = []
for epoch in range(num_epochs):
    # Entrenamiento
    model.train()
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        
    # Evaluación en conjunto de entrenamiento
    model.eval()
    train_output = model(X_train)
    train_loss = criterion(train_output, y_train)
    train_losses.append(train_loss.item())
    
    # Evaluación en conjunto de prueba
    test_output = model(X_test)
    test_loss = criterion(test_output, y_test)
    test_losses.append(test_loss.item())

# Graficar la pérdida durante el entrenamiento y prueba
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss vs. Epoch')
plt.legend()
plt.show()
